In [1]:
import os
import gym

os.unsetenv("SESSION_MANAGER") # Supress rendering issue

env = gym.make("mot_gym:BasicMOT-v0")

ModuleNotFoundError: No module named 'mot_gym.envs.FairMOT.bbox_colors'

In [ ]:
obs, info = env.reset()
print(f"Frame: {info['curr_frame']}, TrackIDs: {info['tracks_ids']}")
env.render()

done = False
while not done: 
    obs, reward, done, info = env.step(1)#env.action_space.sample())
    if info['curr_frame'] % 10 == 0 and info['curr_track']['track_idx'] == 0:
        print(f"Frame: {info['curr_frame']}, TrackIDs: {info['tracks_ids']}")
    env.render()
env.close()

## MOT Gym Demo

In [ ]:
# Env essentials
import os
import gym
import numpy as np

In [ ]:
# Squashs "Qt: Session management error: Could not open network socket" bug
# !echo SESSION_MANAGER - Before
os.unsetenv("SESSION_MANAGER") # - After

### Start by playing with Env
- Test env is working
- Understand action space
- Understand observation space

In [ ]:
# Make env
env = gym.make("mot_gym:BasicMOT-v1")

In [ ]:
# Reset env, should see an observation
obs, _ = env.reset()
# obs = env.reset()
f"Does this have a correct obs space? - {env.observation_space.contains(obs)}"

In [ ]:
# Visualise frame
env.render()

In [ ]:
# Kill open window
env.close()

In [ ]:
# Take multiple steps
for _ in range(5):
    env.step(env.action_space.sample())
    env.render()
env.close()

### Train on MOT17-05

In [ ]:
# RL Lib
import ray
import sys
sys.setrecursionlimit(3000)
from ray.rllib.agents import ppo
from ray.tune.logger import pretty_print

In [ ]:
ray.init()
from ray import rllib
rllib.utils.check_env(env)

In [ ]:
config = ppo.DEFAULT_CONFIG.copy()
config["framework"] = "torch"
config["num_gpus"] = 0
config["num_workers"] = 1

# trainer = ppo.PPOTrainer(config=config, env="test_basic:test-v2")
trainer = ppo.PPOTrainer(config=config, env="mot_gym:BasicMOT-v1")

In [ ]:
# Train using env
for i in range(1):
   # Perform one iteration of training the policy with PPO
   result = trainer.train()
   print(pretty_print(result))

   if i % 10 == 0:
       checkpoint = trainer.save()
       print("checkpoint saved at", checkpoint)

In [ ]:
# Test trainer works
obs, info = env.reset()
print(f"Frame: {info['curr_frame']}, TrackIDs: {info['tracks_ids']}")
env.render()

done = False
while not done:
    action = trainer.compute_single_action(obs)
    obs, reward, done, info = env.step(action)
    if info['curr_frame'] % 10 == 0 and info['curr_track']['track_idx'] == 0:
        print(f"Frame: {info['curr_frame']}, TrackIDs: {info['tracks_ids']}")
    env.render()
env.close()

### Using trainer standalone

In [ ]:
# Excerpt from FairMOT repo